In [1]:
import pandas as pd
import time
import random
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
from unidecode import unidecode 

### Funciones básicas para realizar scraping
#### Funciones para limpiar texto y moneda

In [2]:
#Función para obtener tipo de cambio
def usd():
    #Obtener tipo de cambio
    banxico="https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos/?token=0e825df61e5eca2dd60340f1d39766f5cbefc052fb00f49b257095da3e004921"
    r=requests.get(banxico).json()
    #Obtener último dato
    mxn=r["bmx"]["series"][0]["datos"][-1]["dato"]
    #transformar a float
    mxn=float(mxn)
    return mxn

In [3]:
def limpia_texto(text):
    if text is None:
        return ""
    # Elimina caracteres no alfanuméricos, caracteres, puntuación, espacios extras y signos de pesos
    cleaned_text = re.sub(r'[^\w\s.]', '', text).strip()
    # Minúsculas
    cleaned_text = cleaned_text.lower()
    #Eliminar acentos
    cleaned_text = unidecode(cleaned_text)
    return cleaned_text

def limpia_moneda(text):
    if text is None:
        return ""
    #Eliminar "\n"
    cleaned_coin = re.sub(r'\n', '', text).strip()
    #Elimina comas
    cleaned_coin = re.sub(r',', '', text).strip()
    #Eliminar signo de pesos
    cleaned_coin = re.sub(r'$', '', cleaned_coin)

    return cleaned_coin

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}



def equalize_lists(main_list, *lists):
    main_length = len(main_list)
    for lst in lists:
        while len(lst) < main_length:
            lst.append(None)

In [5]:
def easybroker(municipio, estado,tipo="venta"):
    if tipo == "venta":
        base_url = "https://www.easyaviso.com/mx/inmuebles/propiedades-residenciales-en-venta-en-{}-{}?page={}"
    elif tipo == "renta":
        base_url = "https://www.easyaviso.com/mx/inmuebles/propiedades-residenciales-en-renta-en-{}-{}?page={}"
    elif tipo == "terreno":
        base_url = "https://www.easyaviso.com/mx/inmuebles/terrenos-en-venta-en-{}-{}?page={}"
    else:
        raise ValueError("Selecciona un tipo de propiedad válido: venta, renta o terreno")

    all_data_frames = []

    for page_num in tqdm(range(1, 54), desc=f"Scrapeando Easybroker en {municipio}"):
        url = base_url.format(municipio,estado,page_num)
        r = requests.get(url, headers=headers)
        sopa = BeautifulSoup(r.text, 'html.parser')

        # Cosas a obtener
        recamaras, bathrooms, superficie, direcciones, ofertas, precios, latitud, longitud = [], [], [], [], [], [], [], []

        for precio in sopa.find_all('li', class_='price'):
            precios.append(precio.text.strip())
        for coord in sopa.find_all('li', class_='property__component'):
            latitud.append(coord.get('data-lat'))
            longitud.append(coord.get('data-long'))
        for div in sopa.find_all('div', class_='features'):
            match = re.search(r'(\d+)\s*recámaras', div.text)
            recamaras.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+)\s*baños', div.text)
            bathrooms.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+\.?\d*)\s*m²', div.text)
            superficie.append(float(match.group(1)) if match else None)
        for element in sopa.find_all('div', class_='property__content property-content'):
            direcciones.append(element.find('div', class_='location').text.strip())
        for title in sopa.find_all('a', class_='title'):
            ofertas.append(title.text.strip())

                # Filtrar precios según el tipo de propiedad
        if tipo in ["venta", "terreno"]:
            precios = [price for price in precios if "En Renta" not in price]
        elif tipo == "renta":
            precios = [price for price in precios if "En Venta" not in price]

        equalize_lists(ofertas, recamaras, bathrooms, superficie, direcciones, precios, latitud, longitud)

        data_frame = pd.DataFrame({
            'oferta': ofertas, 'precio': precios, 'mts': superficie,
            'bathrooms': bathrooms, 'recamaras': recamaras, 'lat': latitud,
            'lon': longitud, 'fuente': 'easybroker'
        })
        all_data_frames.append(data_frame)
    combined_df = pd.concat(all_data_frames, ignore_index=True)
    
    if combined_df.empty:
        return combined_df
    
    if tipo in ["venta", "terreno"]:
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Venta", "", regex=False)
                                .str.replace("\n","", regex=False))
    elif tipo == "renta":
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Renta", "", regex=False)
                                .str.replace("\n","", regex=False))
    combined_df = combined_df[~combined_df["precio"].str.contains("por m²")]
    combined_df = combined_df[~combined_df["precio"].str.contains("por ha")]
    combined_df["precio"] = combined_df["precio"].apply(lambda x: float(x.replace("US", "")) * usd() if "US" in x else x)
    combined_df["precio"] = pd.to_numeric(combined_df["precio"], errors="coerce")
    #Eliminar nans
    combined_df = combined_df[combined_df["precio"].notna()]
        #Precio a float
    combined_df["precio"] = combined_df["precio"].astype(float)
        #Eliminar "\n" de precio
    combined_df = combined_df[combined_df["precio"] != 0]
    # Añadir fecha de consulta
    combined_df["fecha_consulta"] = pd.to_datetime("today")
    #Añadir fuente
    combined_df["fuente"] = "easybroker"
    #Añadir municipio
    combined_df["municipio"] = municipio
    #Limpiar oferta
    combined_df["oferta"] = combined_df["oferta"].apply(limpia_texto)
    return combined_df

In [6]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio','recamaras','bathrooms'],keep='first')
    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts']

    return df

#### Realizar webscraping para viviendas en venta

In [7]:
#Scrapear lamudi
viviendas=easybroker("mazatlan","sinaloa","renta")

Scrapeando Easybroker en mazatlan: 100%|██████████| 53/53 [00:34<00:00,  1.56it/s]


In [8]:
vivi_limpia=viviendas.copy()
#Eliminar si oferta dice "lote" o "terreno"
vivi_limpia=vivi_limpia[~vivi_limpia["oferta"].str.contains("lote|terreno")]
#Eliminar si lat es nulo
vivi_limpia=vivi_limpia[vivi_limpia['lat'].notna()]
#Aplicar función de limpieza
vivi_limpia=limpia_datos(vivi_limpia)
vivi_limpia

,oferta,precio,mts,bathrooms,recamaras,lat,lon,fuente,fecha_consulta,municipio,precio_m2
0,departamento en azul marino vacacional mensual,2500.0,NaN,2.0,NaN,23.2990776,-106.4836574,easybroker,2024-09-04 11:20:59.743440,mazatlan,NaN
1,casa en renta en almar residencial,22000.0,163.00,4.0,3.0,23.2961316461,-106.4757102429,easybroker,2024-09-04 11:20:59.743440,mazatlan,134.969325
2,condominio en renta en torre azul,60000.0,283.00,4.0,5.0,23.2176843255,-106.4215564728,easybroker,2024-09-04 11:20:59.743440,mazatlan,212.014134
3,casa en renta en coto diamante,20000.0,110.87,2.0,3.0,23.2904308999,-106.4683770938,easybroker,2024-09-04 11:20:59.743440,mazatlan,180.391449
4,departamento en renta en pacifika marina condos,29000.0,NaN,2.0,3.0,23.2737581,-106.3982246,easybroker,2024-09-04 11:20:59.743440,mazatlan,NaN
...,...,...,...,...,...,...,...,...,...,...,...
88,departamento en renta gavias grand mazatlan,40000.0,NaN,3.0,3.0,23.276905,-106.4674987,easybroker,2024-09-04 11:20:59.743440,mazatlan,NaN
89,departamento en renta en gavias grand luxury c...,35000.0,NaN,NaN,2.0,23.2494148,-106.4111425,easybroker,2024-09-04 11:20:59.743440,mazatlan,NaN
90,departamento en renta en torre eme,35000.0,102.00,2.0,2.0,23.2183102,-106.4220725,easybroker,2024-09-04 11:20:59.743440,mazatlan,343.137255
91,renta casa en excelente zona cerritos,4500.0,152.00,2.0,3.0,23.290961,-106.4768466,easybroker,2024-09-04 11:20:59.743440,mazatlan,29.605263


In [9]:
#guardar csv
vivi_limpia.to_csv("C:/Users/claud/Documents/GitHub/vivi_mazatlan/output/viviendas_mazatlan_easybroker_renta.csv",index=False)